In [9]:
# import packages
import requests
import pandas as pd
from urllib.parse import quote
import time

# state base/headers
BASE = "https://api.ukhsa-dashboard.data.gov.uk"
HEADERS = {"Accept": "application/json"}

In [8]:
# 2024 test
import requests
import pandas as pd
from urllib.parse import quote
import time

BASE = "https://api.ukhsa-dashboard.data.gov.uk"
HEADERS = {"Accept": "application/json"}

TARGET_YEAR = 2024

geo_url = (
    f"{BASE}/themes/immunisation/sub_themes/childhood-vaccines/"
    f"topics/MMR1/geography_types/Upper%20Tier%20Local%20Authority/geographies"
)

geos = requests.get(geo_url, headers=HEADERS).json()
area_names = [g["name"] for g in geos]

vaccines = {
    "MMR1": "MMR1_coverage_coverageByYear",
    "MMR2": "MMR2_coverage_coverageByYear"
}

all_rows = []

for vaccine, metric_name in vaccines.items():
    print(f"Pulling {vaccine}...")
    
    for i, area in enumerate(area_names):
        print(f"{vaccine} | {i+1}/{len(area_names)} | {area}")
        
        encoded_area = quote(area)
        
        url = (
            f"{BASE}/themes/immunisation/sub_themes/childhood-vaccines/"
            f"topics/{vaccine}/geography_types/Upper%20Tier%20Local%20Authority/"
            f"geographies/{encoded_area}/metrics/{metric_name}"
        )
        
        while url:
            r = requests.get(url, headers=HEADERS)
            js = r.json()
            
            for record in js["results"]:
                if record["year"] == TARGET_YEAR:
                    all_rows.append({
                        "area_code": record["geography_code"],
                        "area_name": record["geography"],
                        "year": record["year"],
                        "vaccine": vaccine,
                        "age_stratum": record["stratum"],
                        "coverage": record["metric_value"]
                    })
            
            url = js["next"]
        
        time.sleep(0.1)  # be polite to API

df = pd.DataFrame(all_rows)

df.to_csv("mmr_utla_2024_only.csv", index=False)

print("Done.")

Pulling MMR1...
MMR1 | 1/152 | Barking and Dagenham
MMR1 | 2/152 | Barnet
MMR1 | 3/152 | Barnsley
MMR1 | 4/152 | Bath and North East Somerset
MMR1 | 5/152 | Bedford
MMR1 | 6/152 | Bexley
MMR1 | 7/152 | Birmingham
MMR1 | 8/152 | Blackburn with Darwen
MMR1 | 9/152 | Blackpool
MMR1 | 10/152 | Bolton
MMR1 | 11/152 | Bournemouth, Christchurch and Poole
MMR1 | 12/152 | Bracknell Forest
MMR1 | 13/152 | Bradford
MMR1 | 14/152 | Brent
MMR1 | 15/152 | Brighton and Hove
MMR1 | 16/152 | Bristol, City of
MMR1 | 17/152 | Bromley
MMR1 | 18/152 | Buckinghamshire
MMR1 | 19/152 | Bury
MMR1 | 20/152 | Calderdale
MMR1 | 21/152 | Cambridgeshire
MMR1 | 22/152 | Camden
MMR1 | 23/152 | Central Bedfordshire
MMR1 | 24/152 | Cheshire East
MMR1 | 25/152 | Cheshire West and Chester
MMR1 | 26/152 | Cornwall
MMR1 | 27/152 | County Durham
MMR1 | 28/152 | Coventry
MMR1 | 29/152 | Croydon
MMR1 | 30/152 | Cumberland
MMR1 | 31/152 | Darlington
MMR1 | 32/152 | Derby
MMR1 | 33/152 | Derbyshire
MMR1 | 34/152 | Devon
MMR1 | 

In [10]:
# run across dataset
START_YEAR = 2014
END_YEAR = 2025

# get all upper local authority names
geo_url = (
    f"{BASE}/themes/immunisation/sub_themes/childhood-vaccines/"
    f"topics/MMR1/geography_types/Upper%20Tier%20Local%20Authority/geographies"
)

geos = requests.get(geo_url, headers=HEADERS).json()
area_names = [g["name"] for g in geos]

vaccines = {
    "MMR1": "MMR1_coverage_coverageByYear",
    "MMR2": "MMR2_coverage_coverageByYear"
}

all_rows = []

# function for calling 
for vaccine, metric_name in vaccines.items():
    print(f"\nPulling {vaccine}...")
    
    for i, area in enumerate(area_names):
        print(f"{vaccine} | {i+1}/{len(area_names)} | {area}")
        
        encoded_area = quote(area)
        
        url = (
            f"{BASE}/themes/immunisation/sub_themes/childhood-vaccines/"
            f"topics/{vaccine}/geography_types/Upper%20Tier%20Local%20Authority/"
            f"geographies/{encoded_area}/metrics/{metric_name}"
        )
        
        while url:
            r = requests.get(url, headers=HEADERS)
            js = r.json()
            
            for record in js["results"]:
                
                if START_YEAR <= record["year"] <= END_YEAR:
                    
                    # keeping required strata
                    if vaccine == "MMR1" and record["stratum"] in ["24m", "5y"]:
                        all_rows.append({
                            "area_code": record["geography_code"],
                            "area_name": record["geography"],
                            "year": record["year"],
                            "variable": f"MMR1_{record['stratum']}",
                            "coverage": record["metric_value"]
                        })
                    
                    if vaccine == "MMR2" and record["stratum"] == "5y":
                        all_rows.append({
                            "area_code": record["geography_code"],
                            "area_name": record["geography"],
                            "year": record["year"],
                            "variable": "MMR2_5y",
                            "coverage": record["metric_value"]
                        })
            
            url = js["next"]
        
        time.sleep(0.1)  

# convert to df
df_long = pd.DataFrame(all_rows)

# wide pivot
df_wide = (
    df_long
    .pivot_table(
        index=["area_code", "area_name", "year"],
        columns="variable",
        values="coverage"
    )
    .reset_index()
)

df_wide = df_wide.sort_values(["area_name", "year"])

# export
df_wide.to_csv("mmr_utla_2013_2025_wide.csv", index=False)

print("\nDone.")


Pulling MMR1...
MMR1 | 1/152 | Barking and Dagenham
MMR1 | 2/152 | Barnet
MMR1 | 3/152 | Barnsley
MMR1 | 4/152 | Bath and North East Somerset
MMR1 | 5/152 | Bedford
MMR1 | 6/152 | Bexley
MMR1 | 7/152 | Birmingham
MMR1 | 8/152 | Blackburn with Darwen
MMR1 | 9/152 | Blackpool
MMR1 | 10/152 | Bolton
MMR1 | 11/152 | Bournemouth, Christchurch and Poole
MMR1 | 12/152 | Bracknell Forest
MMR1 | 13/152 | Bradford
MMR1 | 14/152 | Brent
MMR1 | 15/152 | Brighton and Hove
MMR1 | 16/152 | Bristol, City of
MMR1 | 17/152 | Bromley
MMR1 | 18/152 | Buckinghamshire
MMR1 | 19/152 | Bury
MMR1 | 20/152 | Calderdale
MMR1 | 21/152 | Cambridgeshire
MMR1 | 22/152 | Camden
MMR1 | 23/152 | Central Bedfordshire
MMR1 | 24/152 | Cheshire East
MMR1 | 25/152 | Cheshire West and Chester
MMR1 | 26/152 | Cornwall
MMR1 | 27/152 | County Durham
MMR1 | 28/152 | Coventry
MMR1 | 29/152 | Croydon
MMR1 | 30/152 | Cumberland
MMR1 | 31/152 | Darlington
MMR1 | 32/152 | Derby
MMR1 | 33/152 | Derbyshire
MMR1 | 34/152 | Devon
MMR1 |